# Subtitle Remover

Goal was to create a routine to remove Hardcoded subtitles in a Video file.
This uses Open-CV-python or Open Computer Vision for Python

In [2]:
import numpy as np
import cv2

img = cv2.imread("../../subtitleChanger/Sharpe.png")

def hasSubTitle(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mx = np.max(hsv[1])
    return mx >= 255

print(hasSubTitle(img))

True


## Removing the Subtitile

using a Blur mask and a Fill

In [3]:
def removeSubTitle(img, t):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,t,255,cv2.THRESH_BINARY_INV)
    maskblur = cv2.GaussianBlur(thresh, (5,5), 0)
    ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY)
    mask1 = mask
    for i in range(5):
        maskblur = cv2.GaussianBlur(mask, (5,5), 0)
        ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY)

    ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY_INV)
    cv2.imshow("Image", cv2.inpaint(img, mask,3,cv2.INPAINT_TELEA))

img = cv2.imread("../../subtitleChanger/Sharpe.png")
removeSubTitle(img, 240)

error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


## Opening FFMPEG-Pipe

FFMPEG wille be used to open the Video and Stream it Frame by Frame throu the Processor and re compress it afterwards.

In [ ]:
import subprocess as sp

FFMPEG_BIN = "ffmpeg"

def openRead(input:str):
    command = [ FFMPEG_BIN,
                '-i', input,
                '-f', 'image2pipe',
                '-pix_fmt', 'rgb24',
                '-vcodec', 'rawvideo', '-']
    return sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)

def openWrite(output:str, size:str, fps:str):
    command = [ FFMPEG_BIN,
        '-y', # (optional) overwrite output file if it exists
        '-f', 'rawvideo',
        '-vcodec','rawvideo',
        '-s', size, # size of one frame
        '-pix_fmt', 'rgb24',
        '-r', fps, # frames per second
        '-i', '-', # The imput comes from a pipe
        '-an', # Tells FFMPEG not to expect any audio
        output ]

    print(command)

    return sp.Popen( command, stdin=sp.PIPE, stderr=sp.PIPE)

## Multithrading

In [4]:
import tqdm
from threading import Thread

frames = []
threads = []
threadCount = 4

class MTStream:
    def __init__(self, inF, outF):
        self.inFunc = inF
        self.outFunc = outF
        for i in range(threadCount):
            t = Thread(target=self.workThread, args=frames[i])
            threads.append(t)
            frames.append([])
            t.start()

        t = Thread(target=self.combThread, args=self)
        t.start()

    def start(self, arg):
        idx = 0
        for i in tqdm(self.inFunc(arg)):
            frames[idx % threadCount].append({"i": idx, "f":i, "s":False})
            idx += 1

    def workThread(myfrm):
        for i in myfrm:
            if i["s"] is not False:
                if hasSubTitle(i["f"]):
                    removeSubTitle(i["f"])
                i["s"] = True
    
    def combThread(self):
        while frames.count(lambda f: f["s"] == False) > 0:
            pass

        for i in sorted(frames, lambda f: f["i"]):
            self.outFunc(i["f"])
                


7


## Combinding Funktions

In [ ]:
from tqdm import tqdm

sharpeFPS = 25
sharpeWidth = 1280
sharpeHight = 720
sharpeSTThreshold = 240 # threshold: tune this number to your needs
sharpeFile = "./Sharpe.mp4"
sharpeFileOut = "./SharpeNew.mp4"
sharpeLength = 6056

pipeOut = openWrite(sharpeFileOut, f"{sharpeWidth}x{sharpeHight}", f"{sharpeFPS}")

pipeIn = openRead(sharpeFile)
buffSize = sharpeWidth * sharpeHight * 3
frameCount = sharpeLength * sharpeFPS

for i in tqdm(range(frameCount)):
    buff = pipeIn.stdout.read(buffSize)
    image = np.frombuffer(buff, dtype='uint8')
    out = image = np.reshape(image, (sharpeHight, sharpeWidth, 3))

    if hasSubTitle(image):
        out = removeSubTitle(image, sharpeSTThreshold)

    out = np.reshape(out, buffSize)
    pipeOut.stdin.write(out.tobytes())